#### Import dependencies and configure globals
This example, shows how to use gevopy to evolve a simple Haploid genotype in order to obtain a genotype with at least 10 ones in it's chromosome. In the process, all the genotypes are stored inside a neo4j database.

To do so, it is required to import gevopy and generate the Genotype, Fitness and Algorithm required. For the aim of simplicity, we will use some already defined examples from this library. Additionally it is required to import the class Neo4jInterface to specify the connection with our database.

Additionally, you can use neo4j library to verify the connection and the logging module to increase the verbosity of the evolution process.

In [1]:
import logging

import gevopy as ea
from examples.algorimths import BasicPonderated as MyAlgorithm
from examples.evaluation import MostOnes, Random
from examples.genotypes import JackJumper as MyGenotype1
from examples.genotypes import Bacteria as MyGenotype2


In [2]:
LOG_FORMAT = "%(asctime)s - %(name)s - %(levelname)s\n  %(message)s"
logging.basicConfig(level="INFO", format=LOG_FORMAT)
logger = logging.getLogger("gevopy.Experiment")
logger.setLevel(logging.INFO)  # Only Experiment INFO


### Create and run the experiment
Execute an experiment by creating an instance and assigning the argument database with the driver. The experiment will automaticaly manage sessions and use the methods .execute_read and .execute_write.

Use the experiment session as a normal example to evolve a Genotype. As minimalist example, add some genotypes as a base and start the evolution by using .run method.

In [3]:
experiment = ea.Experiment(
    algorithm=MyAlgorithm(survival_rate=0.2),
)

In [4]:
with experiment.session() as session:
    # Add first species genotypes to the session
    session.add_genotypes([MyGenotype1() for _ in range(10)])
    session.set_fitness(MyGenotype1, Random())
    # Add second species genotypes to the session
    session.add_genotypes([MyGenotype2() for _ in range(10)])
    session.set_fitness(MyGenotype2, MostOnes())
    # Start first phase of evolution in session
    session.logger.info("Starting experiment phase 1")
    statistics_1 = session.run(max_generations=10)
    # Replace evaluation of species on 
    session.set_fitness(MyGenotype1, MostOnes())
    # Start second phase of evolution in session
    session.logger.info("Starting experiment phase 2")
    statistics_2 = session.run(max_generations=10)

statistics_2

2023-03-27 11:21:18,005 - gevopy.Experiment - INFO
  [c410ba56-a2c5-429b-bd50-b39f9924669a]: Starting experiment phase 1
2023-03-27 11:21:18,007 - gevopy.Experiment - INFO
  [gen:0]: Start of evolutionary experiment execution
2023-03-27 11:21:18,023 - gevopy.Experiment - INFO
  [gen:1]: Completed cycle; 9
2023-03-27 11:21:18,031 - gevopy.Experiment - INFO
  [gen:2]: Completed cycle; 9
2023-03-27 11:21:18,039 - gevopy.Experiment - INFO
  [gen:3]: Completed cycle; 9
2023-03-27 11:21:18,047 - gevopy.Experiment - INFO
  [gen:4]: Completed cycle; 10
2023-03-27 11:21:18,048 - gevopy.Experiment - INFO
  [gen:4]: Experiment execution completed successfully
2023-03-27 11:21:18,049 - gevopy.Experiment - INFO
  [c410ba56-a2c5-429b-bd50-b39f9924669a]: Starting experiment phase 2
2023-03-27 11:21:18,050 - gevopy.Experiment - INFO
  [gen:0]: Start of evolutionary experiment execution
2023-03-27 11:21:18,061 - gevopy.Experiment - INFO
  [gen:1]: Completed cycle; 0.842471456650234
2023-03-27 11:21:18,

Evolutionary algorithm execution report:
  Executed generations: 2
  Best genotype: f43d92c9-ccdc-4e08-9f0b-17c991037d14
  Best score: 0.9858978453747482

### Close connections
Unless you created them using the with statement, call the .close method on all Driver and Session instances out of the experiment to release any resources still held by them.

In [5]:
experiment.close()
